# <font color='blue'>Preparação conjunto de treinamento e teste</font>

## Montando DRIVE

Corrigir, pelo professor, a próxima célula para o destino correto do Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

drive_dados = '/content/drive/MyDrive/UFSB/Componentes_curriculares/RNA/_02_Regressao/' # Colocar uma barra no final do endereço

## 1) Importando bibliotecas

Importar as bibliotecas e o arquivo que serão utilizadas durante as análises.


In [ ]:
# Imports bilbliotecas básica
# ===========================
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

# Imports bilbliotecas com funções importantes
# ============================================
from sklearn.preprocessing import MinMaxScaler # Normalização [0, 1]
from sklearn.preprocessing import StandardScaler # Normalização

## 2) Extrai os dados do csv ou txt

Um dos grandes problemas que encontramos é mal uso de separador e de encoding, então é importante se atentar a esses argumentos.

É muito importante ressaltar que colocar os argumentos de separador e de encoding quando está lendo um arquivo evita problemas como os dados que não tenham sido separados corretamente.

Fonte dos dados: Dados pre-processados.

In [ ]:
# Carregando o dataset
dataset = pd.read_csv(drive_dados + "Dados_analise_exploratoria.csv", encoding = 'latin-1', sep = ',')
dataset.drop(columns = ["Unnamed: 0"], inplace=True)
print(dataset.shape)
# Verificando as primeiras linhas do dataset
dataset.head()

(954, 9)


,Cimento (X1),Escoria de alto-forno (X2),Cinzas (X3),Agua (X4),Superplastificante (X5),Agregado grosso (X6),Agregado fino (X7),Idade (X8),Forca de compressao do Concreto (Y1)
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,4,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,4,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,11,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,13,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,12,44.296075


Note que não utilizamos o encoding utf-8, pois com o seu uso podemos ter apenas de 0-127 valores, portanto utilzamos o latin-1, que também é conhecido como ISO-8859–1, e que cobre de 0–255 valores.

## 3) Definindo atributos (variáveis de entrada) e targets (variável de saida)

In [ ]:
# Gerando número de observações e variáveis
observations = len(dataset)
variables = dataset.columns
total_variaveis = len(variables)
var_output = 1 # COLOCAR AQUI O NUMERO DE VARIÁVEIS DE SAÍDA (Últimas colunas)
variables_out = dataset.columns[:-var_output]
print('O banco de dados contem %d variáveis.' % total_variaveis)
print(f'Estão registradas {observations} observações.\n')
print(f'Identificadas {total_variaveis-var_output} variáveis preditoras que são:\n{variables[0:total_variaveis-var_output]}\n')
print(f'Identificada(s) {var_output} variáve(l/is) explanatória(s) que é(são):\n{variables[total_variaveis-var_output:]}')

O banco de dados contem 9 variáveis.
Estão registradas 954 observações.

Identificadas 8 variáveis preditoras que são:
Index(['Cimento (X1)', 'Escoria de alto-forno (X2)', 'Cinzas (X3)',
       'Agua (X4)', 'Superplastificante (X5)', 'Agregado grosso (X6)',
       'Agregado fino (X7)', 'Idade (X8)'],
      dtype='object')

Identificada(s) 1 variáve(l/is) explanatória(s) que é(são):
Index(['Forca de compressao do Concreto (Y1)'], dtype='object')


In [ ]:
# Coletando x e y
var_entrada=['Cimento (X1)', 'Escoria de alto-forno (X2)', 'Cinzas (X3)',
       'Agua (X4)', 'Superplastificante (X5)', 'Agregado grosso (X6)',
       'Agregado fino (X7)', 'Idade (X8)'] # Variáveis consideradas na entrada
var_saida=['Forca de compressao do Concreto (Y1)'] # Variáveis consideradas na saida 
dataset = dataset[var_entrada + var_saida]
dataset

,Cimento (X1),Escoria de alto-forno (X2),Cinzas (X3),Agua (X4),Superplastificante (X5),Agregado grosso (X6),Agregado fino (X7),Idade (X8),Forca de compressao do Concreto (Y1)
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,4,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,4,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,11,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,13,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,12,44.296075
...,...,...,...,...,...,...,...,...,...
949,276.4,116.0,90.3,179.6,8.9,870.1,768.3,4,44.284354
950,322.2,0.0,115.6,196.0,10.4,817.9,813.4,4,31.178794
951,148.5,139.4,108.6,192.7,6.1,892.4,780.0,4,23.696601
952,159.1,186.7,0.0,175.6,11.3,989.6,788.9,4,32.768036


## 4) Aplicando Normalização da entrada / saida

### <font color='blue'> Entrada

In [ ]:
# 2) Aplicando Padronização
standardization = StandardScaler()
X_array = standardization.fit_transform(dataset[var_entrada])
dataset[var_entrada] = pd.DataFrame(X_array,columns = dataset[var_entrada].columns).set_index(dataset.index)

### <font color='blue'> Saida 

In [ ]:
# 2) Aplicando Padronização
standardization = StandardScaler()
Y_array = standardization.fit_transform(dataset[var_saida])
dataset[var_saida] = pd.DataFrame(Y_array,columns = dataset[var_saida].columns).set_index(dataset.index)

## 5) Aleatorização do conjunto de dados

In [ ]:
ordem = dataset.sample(dataset.shape[0]).index
dataset = dataset.iloc[ordem]
print(dataset.shape)
dataset

(954, 9)


,Cimento (X1),Escoria de alto-forno (X2),Cinzas (X3),Agua (X4),Superplastificante (X5),Agregado grosso (X6),Agregado fino (X7),Idade (X8),Forca de compressao do Concreto (Y1)
587,-1.491319,1.302470,-0.877906,0.969853,-1.097872,-0.244473,0.365793,0.716281,-0.086204
119,0.440344,1.631729,-0.877906,-1.372023,1.906014,-1.638928,1.395931,1.115191,2.105788
878,-1.191035,0.854631,0.894115,-0.254978,0.582624,-1.440471,0.672397,-0.081538,-0.523182
286,-0.594011,-0.814689,1.862298,-1.416607,1.036357,1.008040,0.061113,-0.480448,-0.052477
678,2.027419,-0.814689,-0.877906,0.406431,-1.097872,-0.536244,-0.648951,-0.081538,0.585694
...,...,...,...,...,...,...,...,...,...
851,-1.172329,-0.814689,1.364563,0.455424,0.792686,-1.311672,1.236856,-0.081538,-1.559150
509,-0.524503,0.879959,-0.877906,0.097773,-1.097872,1.027754,-0.350045,-0.879357,-1.348562
0,2.618142,-0.814689,-0.877906,-1.063366,-0.572717,0.830612,-1.226239,-0.081538,2.852197
165,-1.044831,-0.330240,1.691211,-3.035343,0.103682,1.076383,0.109348,-1.278267,-1.683840


## 6) Divisão do conjunto de dados em treinamento e teste

In [ ]:
# Determinação do percentual do conjunto de treinamento
frac = 0.80 # Fração reservada para a extração do conjunto treinamento
dataset_treinamento = dataset.sample(frac=frac)
dataset_teste = dataset.drop(dataset_treinamento.index)
dataset = dataset_treinamento
print(f'O conjunto de teste tem {dataset_teste.shape}. \nO conjunto de treinamento tem {dataset_treinamento.shape}')

O conjunto de teste tem (191, 9). 
O conjunto de treinamento tem (763, 9)


## 7) Gravando dados

In [ ]:
train_inputs = dataset[var_entrada]
train_targets = dataset[var_saida]
test_inputs = dataset_teste[var_entrada]
test_targets = dataset_teste[var_saida]

In [ ]:
import pickle
# Gravando os dados para o modelo
pickle.dump((train_inputs, train_targets), open(drive_dados + 'dados_treinamento.sav', 'wb'))
pickle.dump((test_inputs, test_targets), open(drive_dados + 'dados_teste.sav', 'wb'))

In [ ]:
train_inputs

,Cimento (X1),Escoria de alto-forno (X2),Cinzas (X3),Agua (X4),Superplastificante (X5),Agregado grosso (X6),Agregado fino (X7),Idade (X8)
584,-1.152638,1.902274,-0.877906,0.406431,-1.097872,-0.544130,0.123332,0.716281
918,0.986763,-0.814689,-0.877906,0.318243,0.309543,0.478384,-0.528362,-0.081538
147,-0.610453,-0.814689,1.029291,-0.166300,0.097380,1.061926,0.099214,-0.081538
169,-1.044831,-0.330240,1.691211,-3.035343,0.103682,1.076383,0.109348,1.514100
349,-1.054184,0.053358,1.740922,-0.963910,0.563718,0.400841,-0.019066,-1.278267
...,...,...,...,...,...,...,...,...
636,-0.694828,2.700093,-0.877906,0.969853,-1.097872,-0.176130,-1.816356,-1.278267
838,0.383241,0.854631,-0.877906,-2.778128,0.582624,0.304898,0.646739,-0.081538
860,-1.132947,1.591435,1.646832,0.847369,0.372562,-1.692814,-1.316040,-0.081538
208,-0.611142,-0.814689,1.078845,-1.210346,0.549014,1.427297,0.358609,0.317371


In [ ]:
train_targets

,Forca de compressao do Concreto (Y1)
584,0.553352
918,0.282022
147,-0.606655
169,0.293019
349,-1.195903
...,...
636,-1.570060
838,0.625093
860,-0.231286
208,0.660595
